# 1. Crawler

## 1.0. Related example

This code shows `wget`-like tool written in python. Run it from console (`python wget.py`), make it work. Check the code, reuse, and modify for your needs.

In [7]:
import argparse
import os
import re
import requests


def wget(url, filename):
    # allow redirects - in case file is relocated
    resp = requests.get(url, allow_redirects=True)
    # this can also be 2xx, but for simplicity now we stick to 200
    # you can also check for `resp.ok`
    if resp.status_code != 200:
        print(resp.status_code, resp.reason, 'for', url)
        return
    
    # just to be cool and print something
    print(*[f"{key}: {value}" for key, value in resp.headers.items()], sep='\n')
    print()
    
    # try to extract filename from url
    if filename is None:
        # start with http*, ends if ? or # appears (or none of)
        m = re.search("^http.*/([^/\?#]*)[\?#]?", url)
        filename = m.group(1)
        if not filename:
            raise NameError(f"Filename neither given, nor found for {url}")

    # what will you do in case 2 websites store file with the same name?
    if os.path.exists(filename):
        raise OSError(f"File {filename} already exists")
    
    with open(filename, 'wb') as f:
        f.write(resp.content)
        print(f"File saved as {filename}")


if __name__ == "__main__":
    # parser = argparse.ArgumentParser(description='download file.')
    # parser.add_argument("-O", type=str, default=None, dest='filename', help="output file name. Default -- taken from resource")
    # parser.add_argument("url", type=str, default=None, help="Provide URL here")
    # args = parser.parse_args()
    # wget(args.url, args.filename)
    url = "https://github.com"
    filename = "github.html"
    wget(url, filename)
    

Server: GitHub.com
Date: Sun, 06 Feb 2022 23:18:30 GMT
Content-Type: text/html; charset=utf-8
Vary: X-PJAX, X-PJAX-Container, Accept-Language, Accept-Encoding, Accept, X-Requested-With
permissions-policy: interest-cohort=()
content-language: en-US
ETag: W/"de731bb6292e06c47193421ab93ab6e1"
Cache-Control: max-age=0, private, must-revalidate
Strict-Transport-Security: max-age=31536000; includeSubdomains; preload
X-Frame-Options: deny
X-Content-Type-Options: nosniff
X-XSS-Protection: 0
Referrer-Policy: origin-when-cross-origin, strict-origin-when-cross-origin
Expect-CT: max-age=2592000, report-uri="https://api.github.com/_private/browser/errors"
Content-Security-Policy: default-src 'none'; base-uri 'self'; block-all-mixed-content; child-src github.com/assets-cdn/worker/ gist.github.com/assets-cdn/worker/; connect-src 'self' uploads.github.com objects-origin.githubusercontent.com www.githubstatus.com collector.githubapp.com collector.github.com api.github.com github-cloud.s3.amazonaws.com 

### 1.0.1. How to parse a page?

If you build a crawler, you might follow one of the approaches:
1. search for URLs in the page, assuming this is just a text.
2. search for URLs in the places where URLs should appear: `<a href=..`, `<img src=...`, `<iframe src=...` and so on.

To follow the first approach you can rely on some good regular expression. [Like this](https://stackoverflow.com/a/3809435).

To follow the second approach just read one of these: [short answer](https://stackoverflow.com/questions/1080411/retrieve-links-from-web-page-using-python-and-beautifulsoup) or [exhaustive explanation](https://hackersandslackers.com/scraping-urls-with-beautifulsoup/).

## 1.1. [15] Download and persist #
Please complete a code for `load()`, `download()` and `persist()` methods of `Document` class. What they do:
- for a given URL `download()` method downloads binary data and stores in `self.content`. It returns `True` for success, else `False`.
- `persist()` method saves `self.content` somewhere in file system. We do it to avoid multiple downloads (for caching in other words).
- `load()` method loads data from hard drive. Returns `True` for success.

Tests checks that your code somehow works.

**NB Passing the test doesn't mean you correctly completed the task.** These are **criteria, which have to be fullfilled**:
1. URL is a unique identifier (as it is a subset of URI). Thus, documents with different URLs should be stored in different files. Typical errors: documents from the same domain are overwritten to the same file, URLs with similar endings are downloaded to the same file, etc.
2. The document can be not only a text file, but also a binary. Pay attention that if you download `mp3` file, it still can be played. Hint: don't hurry to convert everything to text.

In [8]:
import requests
from urllib.parse import quote

class Document:
    
    def __init__(self, url):
        self.url = url
        self.filename = None
        self.content = ''

    def get(self):
        if not self.load():
            if not self.download():
                raise FileNotFoundError(self.url)
            else:
                self.persist()
    
    def download(self):
        #TODO download self.url content, store it in self.content and return True in case of success
        page = requests.get(self.url, allow_redirects=True)
        if page.ok:
            self.content = page.content
            return True
        return False
    
    def persist(self):
        #TODO write document content to hard drive
        if self.filename is None:
            # m = re.search("^http.*/([^/\?#]*)[\?#]?", self.url)
            # self.filename = m.group(1)
            # if not self.filename:
            #     m = re.search("^http.*/([^]*)?", self.url)
            #     self.filename = m.group(1)
            self.filename = self.url.replace('https://', '').replace('http://', '').replace('/', '_')
            if not self.filename:
                raise NameError(f"Filename neither given, nor found for {self.url}")

        if os.path.exists(self.filename):
            raise OSError(f"File {self.filename} already exists")
            
        with open(self.filename, 'wb') as f:
            f.write(self.content)
        pass
            
    def load(self):
        #TODO load content from hard drive, store it in self.content and return True in case of success
        try:
            with open(self.filename, 'rb') as f:
                self.content = f.read()
            return True
        except Exception:
            return False

### 1.1.1. Tests ###

In [10]:
doc = Document('http://sprotasov.ru/data/iu.txt')

doc.get()
assert doc.content, "Document download failed"
assert "Code snippets, demos and labs for the course" in str(doc.content), "Document content error"

doc.get()
assert doc.load(), "Load should return true for saved document"
assert "Code snippets, demos and labs for the course" in str(doc.content), "Document load from disk error"

## 1.3. [10] Parse HTML ##
`BeautifulSoap` library is a de facto standard to parse XML and HTML documents in python. Use it to complete `parse()` method that extracts document contents. You should initialize:
- `self.anchors` list of tuples `('text', 'url')` met in a document. Be aware, there exist relative links (e.g. `../content/pic.jpg`). Use `urllib.parse.urljoin()` to fix this issue.
- `self.images` list of images met in a document. Again, links can be relative to current page.
- `self.text` should keep plain text of the document without scripts, tags, comments and so on. You can refer to [this stackoverflow answer](https://stackoverflow.com/a/1983219) for details.

**NB All these 3 criteria must be fulfilled to get full point for the task.**

In [11]:
from bs4 import BeautifulSoup
from bs4.element import Comment
import urllib.parse


class HtmlDocument(Document):
    
    def tag_visible(self, element):
        if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
            return False
        if isinstance(element, Comment):
            return False
        return True

    def parse(self):
        #TODO extract plain text, images and links from the document
        self.anchors = []
        self.images = []
        self.text = ""

        body = urllib.request.urlopen(self.url).read()
        soup = BeautifulSoup(body, 'html.parser')

        for link in soup.findAll('a'):
            if link['href'] != '' and link['href'] != '#':
                if ((link.text, link['href'])) not in self.anchors:
                    if self.url not in link['href']:
                        self.anchors.append((link.text, urllib.parse.urljoin(self.url, link['href'])))
                    else:    
                        self.anchors.append((link.text, link['href']))

        images = soup.findAll('img')
        for image in images:
            if image['src'] not in self.images:
                if self.url not in image['src']:
                    self.images.append(urllib.parse.urljoin(self.url, image['src']))
                else:
                    self.images.append(image['src'])

        texts = soup.findAll(text=True)
        visible_texts = filter(self.tag_visible, texts)  
        self.text = u" ".join(t.strip() for t in visible_texts)


    

### 1.3.1. Tests ###

In [12]:
doc = HtmlDocument("http://sprotasov.ru")
doc.get()
doc.parse()

assert "just few links" in doc.text, "Error parsing text"
assert "http://sprotasov.ru/images/gb.svg" in doc.images, "Error parsing images"
assert any(p[1] == "https://twitter.com/07C3" for p in doc.anchors), "Error parsing links"

## 1.4. [10] Document analysis ##
Complete the code for `HtmlDocumentTextData` class. Implement word and sentence splitting (use any method you can propose). 

**Criteria of success**: 
1. Your `get_word_stats()` method should return `Counter` object.
2. Don't forget to lowercase your words for counting.
3. Sentences should be obtained inside `<body>` tag only.

In [13]:
from collections import Counter

class HtmlDocumentTextData:
    
    def __init__(self, url):
        self.doc = HtmlDocument(url)
        self.doc.get()
        self.doc.parse()
    
    def get_sentences(self):
        #TODO implement sentence parser
        result = []
        plain_text = self.doc.text.split(' ')
        for word in plain_text:
            if word != '' and word != '-':
                 if not '.' in word:
                    result.append(word.lower())
        return result
    
    def get_word_stats(self):
        result = self.get_sentences()
        return Counter(result)

### 1.4.1. Tests ###

In [14]:
doc = HtmlDocumentTextData("https://innopolis.university/")

print(doc.get_word_stats().most_common(10))
assert [x for x in doc.get_word_stats().most_common(10) if x[0] == 'иннополис'], 'иннополис should be among most common'

[('и', 59), ('в', 30), ('иннополис', 19), ('по', 16), ('на', 14), ('университет', 12), ('области', 10), ('с', 10), ('лаборатория', 10), ('университета', 9)]


## 1.5. [15] Crawling ##

Method `crawl_generator()` is given starting url (`source`) and max depth of search. It should return a **generator** of `HtmlDocumentTextData` objects (return a document as soon as it is downloaded and parsed). You can benefit from `yield obj_name` python construction. Use `HtmlDocumentTextData.anchors` field to go deeper.

In [18]:
from queue import Queue

class Crawler:
    
    def crawl_generator(self, source, depth=1):
        #TODO return real crawling results. Don't forget to process failures

        visited: list() = list()
        depth_link = [(source, 0)]

        while len(depth_link) != 0:
            link = depth_link.pop(0)
            if link[1] + 1 >= depth:
                continue
            visited.append(link[0])
            anchors = HtmlDocumentTextData(link[0]).doc.anchors
            for anchor in anchors:
                if anchor[1] in visited:
                    continue
                try:
                    yield HtmlDocumentTextData(anchor[1])
                    depth_link.append((anchor[1], link[1] + 1))
                    visited.append(anchor[1])
                except:
                    print("Skipping", anchor[1])
    

### 1.5. Tests ###

In [20]:
crawler = Crawler()
counter = Counter()

for c in crawler.crawl_generator("https://innopolis.university/en/", 2):
    print(c.doc.url)
    if c.doc.url[-4:] in ('.pdf', '.mp3', '.avi', '.mp4', '.txt'):
        print("Skipping", c.doc.url)
        continue
    counter.update(c.get_word_stats())
    print(len(counter), "distinct word(s) so far")
    
print("Done")

print(counter.most_common(20))
assert [x for x in counter.most_common(20) if x[0] == 'innopolis'], 'innopolis sould be among most common'

Skipping https://apply.innopolis.university/en
Skipping https://corporate.innopolis.university/en
https://media.innopolis.university/en
306 distinct word(s) so far
https://innopolis.university/lk/
680 distinct word(s) so far
https://innopolis.university/en/about/
1001 distinct word(s) so far
https://innopolis.university/en/board/
1095 distinct word(s) so far
https://innopolis.university/en/team/
1097 distinct word(s) so far
https://innopolis.university/en/team-structure/
1102 distinct word(s) so far
https://innopolis.university/en/team-structure/education-academics/
1108 distinct word(s) so far
https://innopolis.university/en/team-structure/techcenters/
1111 distinct word(s) so far
Skipping https://innopolis.university/en/faculty/
Skipping https://innopolis.university/en/faculty/
Skipping https://career.innopolis.university/en/job/
Skipping https://career.innopolis.university/en/
Skipping https://innopolis.university/en/campus
https://innopolis.university/en/contacts/
1121 distinct wor

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


https://innopolis.university/public/files/Consent_to_the_processing_of_PD_for_UI.pdf
Skipping https://innopolis.university/public/files/Consent_to_the_processing_of_PD_for_UI.pdf
Done
[('and', 1445), ('of', 1253), ('the', 883), ('in', 572), ('to', 388), ('for', 339), ('university', 320), ('lab', 303), ('research', 286), ('и', 282), ('development', 251), ('science', 241), ('software', 224), ('innopolis', 220), ('data', 207), ('education', 191), ('it', 181), ('robotics', 178), ('artificial', 175), ('a', 164)]
